# BMIN 5200 Foundations of Artificial Intelligence in Health

Assignment 3: Expert Systems in clipspy

In [1]:
import clips
import sys
sys.path.append('./src/')
from clips_util import print_facts, print_rules, print_templates, build_read_assert

In [2]:
# create the CLIPS environment
env = clips.Environment()

# Endometriosis Classification System

Endometriosis Classification Algorithm:
Using clipspy to implement an expert system that classifies a patient into one of the following categories:

Outputs:
1. Endometriosis
2. Not Endometriosis (some other disease causing symptoms)
3. Endometriosis and another concomitant disease

![resource](reference/clinical_diagnosis_algo.png)

# Templates

## Part 1: Defining Fact Templates (that are consistent with endometriosis)

This section defines and builds all of the templates that are needed to track and evalute facts associated with inclusion criteria.

In [ ]:
# basic patient information
DEFTEMPLATE_PATIENT = """
(deftemplate patient 
    (slot name_is (type STRING))
    (slot age_is (type INTEGER))
    )
"""
env.build(DEFTEMPLATE_PATIENT)

# participant pelvic pain
DEFTEMPLATE_ACUTE_PELVIC_PAIN = """
(deftemplate pelvic_pain
    (slot has_pain (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_ACUTE_PELVIC_PAIN)

# participant Karnofsky quality of life score
DEFTEMPLATE_DYSMENORRHEA = """
(deftemplate dysmenorrhea
  (slot has_dysmenorrhea (type SYMBOL)
    (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_DYSMENORRHEA)

# participant HIV ELISA test result
DEFTEMPLATE_DYSPAREUNIA = """
(deftemplate dyspareunia
    (slot has_dyspareunia (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_DYSPAREUNIA)

# participant dyschezia status
DEFTEMPLATE_DYSCHEZIA = """
(deftemplate dyschezia
   (slot has_dyschezia (type SYMBOL)
        (allowed-symbols yes no unknown)))
"""
env.build(DEFTEMPLATE_DYSCHEZIA)

# participant dysuria status
DEFTEMPLATE_DYSURIA = """
(deftemplate dysuria
   (slot has_dysuria (type SYMBOL)
        (allowed-symbols yes no unknown)))
"""
env.build(DEFTEMPLATE_DYSURIA)

# participant pregnancy status
DEFTEMPLATE_INFERTILITY = """
(deftemplate infertility    
    (slot is_infertile (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_INFERTILITY)

# participant chronic pelvic pain
DEFTEMPLATE_CHRONIC_PELVIC_PAIN = """
(deftemplate chronic_pelvic_pain
    (slot has_chronic_pelvic_pain (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_CHRONIC_PELVIC_PAIN)

## Part 2: Defining Fact Templates (factors indicating to consider another diagnossis in addition to OR instead of endometriosis)
Consistent with / not consistent wtih endo Fact Templates

In [ ]:
# participant has amenorrhea
DEFTEMPLATE_AMENORRHEA = """
(deftemplate amenorrhea
    (slot has_amenorrhea (type SYMBOL) 
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_AMENORRHEA)

# participant constipation status
DEFTEMPLATE_CONSTIPATION = """
(deftemplate constipation
    (slot has_constipation (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_CONSTIPATION)

# participant diarrhea status
DEFTEMPLATE_DIARRHEA = """
(deftemplate diarrhea
    (slot has_diarrhea (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_DIARRHEA)

# participant flank pain status
DEFTEMPLATE_FLANK_PAIN = """
(deftemplate flank_pain
    (slot has_flank_pain (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_FLANK_PAIN)

# participant hematuria status
DEFTEMPLATE_HEMATURIA = """
(deftemplate hematuria
    (slot has_hematuria (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_HEMATURIA)

# participant frequent urination status
DEFTEMPLATE_FREQUENT_URINATION = """
(deftemplate frequent_urination
    (slot has_frequent_urination (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_FREQUENT_URINATION)

# participant adenomyosis status
DEFTEMPLATE_ADENOMYOSIS = """
(deftemplate adenomyosis
    (slot has_adenomyosis (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_ADENOMYOSIS)

## Consistent with / not consistent with endo Fact Templates
This section defines and builds all of the templates that are needed to track and evalute whether inclusion, exclusion, criteria for endometriosis and/or concomitant diseases.

In [ ]:
# participant inclusion criteria met status
DEFTEMPLATE_ENDOMETRIOSIS_INCLUSION = """
(deftemplate endometriosis_inclusion
    (slot meets_criteria (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_ENDOMETRIOSIS_INCLUSION)

# participant trial eligibility status
DEFTEMPLATE_CONCOMITANT_DISEASE_INCLUSION = """
(deftemplate concomitant_disease_inclusion
    (slot meets_criteria (type SYMBOL)
    (allowed-symbols yes no unknown possible))
)
"""
env.build(DEFTEMPLATE_CONCOMITANT_DISEASE_INCLUSION)

# Initialize Knowledge Base

This section adds the necessary `deffacts`. These include four facts with slot values set to `unknown` to prevent some rules from firing at inappropriate times. As different rules are executed and facts asserted, these slot values will be updated so that, through chaining, other rules will then fire as needed. You should review the already implemented `defrules` carefully to understand how these work.

In [ ]:
# Add deffacts that the inclusion, exclusion, trial eligibility, and child bearing status are all unknown
DEFFCATS_INITIAL_STATUS = """
(deffacts starting_inclusion_exclusion_facts "Set the inclusion criteria met to unknown"
    (endometriosis_inclusion (meets_criteria unknown))
    (concomitant_disease_inclusion (meets_criteria unknown))
)
"""
env.build(DEFFCATS_INITIAL_STATUS)

# reset the environment to make sure the deffacts are added
env.reset()

# Inference Rules

## Inclusion Criteria Rules

The `defrules` in this section support gathering user input about the patient relative to the inclusion criteria and determining if the inclusion criteria are met. Notice that if the initial criteria (age, Karnofsky quality of life, life expectancy, and HIV status) are not met, no further information is collected as it is already known that trial eligibility will not be satisfied. If the initial inclusion criteria are met, then the value of the patient's sex is requested. If the patient is female, yet more information is requested.

In [ ]:
# These are the prompts that will be displayed to the user when requesting various template:slot inputs.
prompt_map = {
    "patient:name_is": "Enter patient name: ",
    "patient:age_is": "Enter patient age (in years): ",
    "pelvic_pain:has_pain": "Does the patient have acute pelvic pain (yes or no or unknown)?: ",
    "dysmenorrhea:has_dysmenorrhea": "Does the patient have dysmenorrhea (yes or no or unknown)?: ",
    "dyspareunia:has_dyspareunia": "Does the patient have dyspareunia (yes or no or unknown)?: ",
    "dyschezia:has_dyschezia" : "Does the patient have dyschezia (yes or no or unknown)?: ",
    "dysuria:has_dysuria" : "Does the patient have dysuria (yes or no or unknown)?: ",
    "infertility:is_infertile" : "Is the patient infertile (yes or no or unknown)?: ",
    "chronic_pelvic_pain:has_chronic_pelvic_pain" : "Does the patient have chronic pelvic pain (peritonitis) (yes or no or unknown)?: ",
    "amenorrhea:has_amenorrhea" : "Does the patient have amenorrhea (yes or no or unknown)?: ",
    "constipation:has_constipation" : "Does the patient have constipation related to IBS (yes or no or unknown)?: ",
    "diarrhea:has_diarrhea" : "Does the patient have diarrhea (yes or no or unknown)?: ",
    "flank_pain:has_flank_pain" : "Does the patient have flank pain (yes or no or unknown)?: ",
    "hematuria:has_hematuria" : "Does the patient have hematuria (yes or no or unknown)?: ",
    "frequent_urination:has_frequent_urination" : "Does the patient have frequent urination (yes or no or unknonw)?: ", 
    "adenomyosis:has_adenomyosis" : "Does the patient have adenomyosis (yes or no or unknown)?: "
}
build_read_assert(env, prompt_map) # see src/clips_util.py for implementation and demos/clips/clips_basics.ipynb for usage

DEFRULE_READ_INITIAL_INCLUSION = """
(defrule reading_input_initial
    =>
    (read_assert patient)
    (read_assert life_expect)
    (read_assert karnofsky)
    (read_assert hiv_testing)
)
"""
env.build(DEFRULE_READ_INITIAL_INCLUSION)

# Determine child bearing age status using forward chaining based on female sex and age
# Only want this to run if the female candidate is not already excluded based on other criteria.
DEFRULE_CHILD_BEARING_AGE = """
(defrule child_bearing_eval "Evaluate for child_bearing age or not for possibly eligible females"
    (logical
    	(patient (age_is ?age) (sex_is female))
    	(test(>= ?age 18))
    	(test(< ?age 45)) ;commonly accepted standard of upper limit of child bearing age of 45
    	
    	(life_expect (years_expected_is ?life_expected))
       (karnofsky (performance_status_is ?karnofsky_score))
    	(test (>= ?life_expected 1))
      	(test (>= ?karnofsky_score 60))
 		(hiv_testing (hiv_elisa_is positive))
 	)
    
    =>
   
    (assert (child_bearing(of_child_bearing_age yes)))
)
"""
env.build(DEFRULE_CHILD_BEARING_AGE)


# Get additional information from user if it is determined that participant is a child bearing age female. 
# Forward chaining to infer whether a female is still eligible by not being pregnant and having adequate pregnancy prevention
DEFRULE_INPUT_PREG_EVAL = """
(defrule reading_input_preg_eval "Ask for data on patient_pregnancy evaluation and birth control"
        (child_bearing(of_child_bearing_age yes))

        =>
        
        (read_assert serum_preg)

)
"""
env.build(DEFRULE_INPUT_PREG_EVAL)


# Get additional information from user if it is determined that participant is of child bearing age, female, and not already pregnant
# Supports forward chaining to infer whether a female is still eligible by not being pregnant and having adequate pregnancy prevention
DEFRULE_INPUT_BC = """
(defrule reading_input_birthcontrol_eval "Ask for data on birth control if the patient is of child-bearing age and not pregnant"
    (child_bearing(of_child_bearing_age yes))
    (serum_preg (preg_test_is negative))
    
    =>
    
    ;(printout t "Is the patient on birth control (yes or no)?: ")
    ;(assert (birth-control (on-birth-control (read))))

    (read_assert birth_control)
)
"""
env.build(DEFRULE_INPUT_BC)

# ; RULE: Inclusion Criteria Are Not Met
# ; *Forward chaining to determine if participant is not eligible for study based on inclusion criteria facts
# ; INPUT: Criteria based on inclusion criteria defined in study. 
# ; OUTPUT: Trial eligibility No, Inclusion Criteria Met No
DEFRULE_INCLUSION_CRITERIA_NOT_MET = """
(defrule inclusion-criteria-notmet "Rule to define a person as not eligible for study based on inclusion criteria facts"
    (logical
        (patient (age_is ?age))
        (life_expect (years_expected_is ?life_expected))
        (karnofsky (performance_status_is ?karnofsky_score))
        (or  ; An OR connector for any of the reasons that an inclusion is not met.
        	  (test (not (>= ?age 18)))  ; Age not greater than or equal to 18
            (hiv_testing (hiv_elisa_is ~positive)) ; Not Positive HIV test
            (test (not (>= ?life_expected 1))) ; Life expectancy Not greater than or equal to 1
            (test (not (>= ?karnofsky_score 60))) ; Karnofsky score not greater than or equal to 60
            
            ;(female sex) and (of child bearing age) and ( (preg not neg) or (birth control not yes))
            
            (and 
                (patient (sex_is female)) 
                (child_bearing(of_child_bearing_age yes))
                
                (or
                    (serum_preg (preg_test_is ~negative))
                    (birth_control (on_birth_control ~yes))        
                )
            )
        )    
    )

    ?f1 <-(hiv_trial_inclusion (meets_criteria unknown))
    ?f2 <-(inclusion (criteria_met unknown))

    => 

    (modify ?f1 (meets_criteria no))
    (modify ?f2 (criteria_met no))
)
"""
env.build(DEFRULE_INCLUSION_CRITERIA_NOT_MET)



SyntaxError: unterminated string literal (detected at line 14) (1162603327.py, line 14)

### Problem 1: 6 points

In the code cell below, implement a clips `defrule` that fires if __all__ of the inclusion criteria are met. It should update the value of the slot `meets_criteria` to `possible` in the fact `hiv_trial_inclusion` and the value of the slot `criteria_met` to `yes` in the fact `inclusion`.  _HINT_: This rule will look very similar to the `DEFRULE_INCLUSION_CRITERIA_NOT_MET` defined in the code cell above.

In [ ]:
#######  YOUR CODE HERE ##########
DEFRULE_INCLUSION_CRITERIA_MET = """
(defrule inclusion-criteria-met "Rule to define a person as POSSIBLY eligible for study based on inclusion criteria facts"
    (logical
        (patient (age_is ?age))
        (life_expect (years_expected_is ?life_expected))
        (karnofsky (performance_status_is ?karnofsky_score))
        (or 
            (and  ; An AND connector for any of the reasons that an inclusion is not met.
                (test (>= ?age 18))  ; Age greater than or equal to 18
                (hiv_testing (hiv_elisa_is positive)) ; Positive HIV test
                (test (>= ?life_expected 1)) ; Life expectancy greater than or equal to 1
                (test (>= ?karnofsky_score 60)) ; Karnofsky score greater than or equal to 60
                
                ;(female sex) and (of child bearing age) and ( (preg neg) and (birth control yes))
                
                    (and 
                        (patient (sex_is female)) 
                        (child_bearing(of_child_bearing_age yes))
                        
                        (and
                            (serum_preg (preg_test_is negative))
                            (birth_control (on_birth_control yes))        
                        )
                    )
                )

            (and  ; An AND connector for any of the reasons that an inclusion is not met (if the patient is male).
                (test (>= ?age 18))  ; Age greater than or equal to 18
                (hiv_testing (hiv_elisa_is positive)) ; Positive HIV test
                (test (>= ?life_expected 1)) ; Life expectancy greater than or equal to 1
                (test (>= ?karnofsky_score 60)) ; Karnofsky score greater than or equal to 60
                (patient (sex_is male))
            )
        ) 
    )

    ?f1 <-(hiv_trial_inclusion (meets_criteria unknown))
    ?f2 <-(inclusion (criteria_met unknown))

    => 

    (modify ?f1 (meets_criteria possible))
    (modify ?f2 (criteria_met yes))
)
"""
#######  END CODE HERE ##########
env.build(DEFRULE_INCLUSION_CRITERIA_MET)

## Inclusion Criteria Status Reporting Rules

The cell below defines `defrules` that fire once it is determine if the inclusion criteria are satisfied or not to report the status to the user.

In [ ]:
DEFRULE_POSSIBLE_ELIGIBILITY = """
(defrule possibly-eligible
    (inclusion (criteria_met yes))
    =>
    (println "___________")
    (println "All inclusion criteria are met, this patient is POSSIBLY ELIGIBLE for the study.")
    (println "___________")
    (assert (start_exclusion yes)) 
)
"""
env.build(DEFRULE_POSSIBLE_ELIGIBILITY)

DEFRULE_NOT_ELIGIBLE_INCLUSION = """
(defrule not-eligible-inclusion
    (inclusion (criteria_met no))
    =>
    (println "___________")
    (println "Inclusion crtieria are NOT met. This patient is NOT ELIGIBLE for the study")
    (println "___________")
)
"""
env.build(DEFRULE_NOT_ELIGIBLE_INCLUSION)

## Exclusion Criteria Rules

The `defrules` in this section support gathering user input about the patient relative to the exclusion criteria and determining if the exclusion criteria are met. 

In [ ]:
# Get addditional information related to exclusion criteria after determining if the participant is possibly eligible for the study 
# possibly eligible = meets all inclusion criteria, fact determined through forward chaining above
DEFRULE_READ_INPUT_EXCLUSION = """
(defrule reading-input-exclusion "Get extra information if inclusion criteria are complete regarding exclusion criteria"
    (inclusion (criteria_met yes))
    (start_exclusion yes) ; This ensures reporting possibly eligible occurs before starting exclusion intake questions
    (exclusion (criteria_met unknown))
    
    =>
    
    (read_assert other_study)
    (read_assert any_drugs)
    (read_assert hemoglobin)
    (read_assert neutrophil)

)
"""
env.build(DEFRULE_READ_INPUT_EXCLUSION)

# ; RULE: Exclusion Criteria Are Met
# ; *Forward chaining to determine if participant with fact of possible eligibility is actually not eligible for study based on exclusion criteria facts
# ; INPUT: Criteria based on exclusion criteria defined in study + Inclusion Criteria are met
# ; OUTPUT: Trial eligibility No, Exclusion Criteria Met Yes (If exclusion criteria are met, then the participant is not eligible for the study)
DEFRULE_EXCLUSION_CRITERIA_MET = """
(defrule exclusion_criteria_met_check  "Rule to check if any exclusion criteria are  met. If they are  met then the patient is NOT eligible"
    (logical
        (inclusion (criteria_met yes))
        (hemoglobin (hemoglobin_count_is ?hemoglobin_count))
        (neutrophil (neutrophil_count_is ?neutrophil_count))
        
        (or (other_study(is_other_study_participant yes))
            (any_drugs(has_taken_drugs yes))
            (test (< ?hemoglobin_count 8.6))
            (test (< ?neutrophil_count 1000))
        )
    )

    ?f1 <-(hiv_trial_inclusion (meets_criteria possible))
    ?f2<-(exclusion (criteria_met unknown))
 
    => 
    
    (modify ?f1 (meets_criteria no))
    (modify ?f2 (criteria_met yes))
)
"""
env.build(DEFRULE_EXCLUSION_CRITERIA_MET)


In [ ]:
# ; RULE: Exclusion Criteria Are Not Met
# ; *Forward chaining to determine if participant with fact of possible eligibility is actually eligible for study based on exclusion criteria facts
# ; INPUT: Criteria based on exclusion criteria defined in study NOT met + Inclusion Criteria are met
# ; OUTPUT: Trial eligibility YES, Exclusion Criteria Met No (If exclusion criteria are not met, then the participant is eligible for the study)
#######  YOUR CODE HERE ##########
DEFRULE_EXCLUSION_CRITERIA_NOT_MET = """
(defrule exclusion_criteria_not_met_check  "Rule to check if any exclusion criteria are  met. If they are not met then the patient IS eligible"
    (logical
        (inclusion (criteria_met yes))
        (hemoglobin (hemoglobin_count_is ?hemoglobin_count))
        (neutrophil (neutrophil_count_is ?neutrophil_count))
        
        (and (other_study(is_other_study_participant no))
            (any_drugs(has_taken_drugs no))
            (test (not (< ?hemoglobin_count 8.6)))
            (test (not (< ?neutrophil_count 1000)))
        )
    )

    ?f1 <-(hiv_trial_inclusion (meets_criteria possible))
    ?f2<-(exclusion (criteria_met unknown))
 
    => 
    
    (modify ?f1 (meets_criteria yes))
    (modify ?f2 (criteria_met no))
)
"""
#######  END CODE HERE ##########
env.build(DEFRULE_EXCLUSION_CRITERIA_NOT_MET)

## Exclusion Criteria Reporting

The cell below defines `defrules` that fire if it is determined that exclusion criteria are met to report the status to the user.

In [ ]:
DEFRULE_NOT_ELIGIBLE_EXCLUSION = """
(defrule not_eligible_exclusion
    (exclusion (criteria_met yes))
    (hiv_trial_inclusion (meets_criteria no))
    =>
    (println "___________")
    (println "All inclusion criteria are met BUT 1 or more exclusion criteria are met. This patient is NOT ELIGIBLE for the study.")
    (println "___________")
)
"""
env.build(DEFRULE_NOT_ELIGIBLE_EXCLUSION)

In [ ]:
#######  YOUR CODE HERE ##########
DEFRULE_TRIAL_ELIGIBLE = """
(defrule trial_eligible
    (exclusion (criteria_met no))
    (hiv_trial_inclusion (meets_criteria yes))
    =>
    (println "___________")
    (println "All inclusion criteria are met and none of the exclusion criteria are met. This patient is ELIGIBLE for the study!")
    (println "___________")
)
"""
#######  END CODE HERE ##########
env.build(DEFRULE_TRIAL_ELIGIBLE)

# Execution
You should use the cells below to test the system behavior while implementing the `defrules` in this assignment. 

In [ ]:
env.reset();
env.run();

In [ ]:
print_facts(env)

## Evaluating Expert System Performance

In [ ]:
# three outputs
# calculate accuracy, PPV, NPV for each case

# and ablation analysis

# input data path: /project/ssverma_shared/projects/Endometriosis/Endo_RuleBased_Phenotyping/symptom_pulls/Pheno/PMBB_2.3_pheno_covars.csv